In [14]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from sklearn import preprocessing, tree
from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import KFold

In [15]:
df=pd.read_json('../01_Preprocessing/First.json').sort_index()

In [16]:
df.head(2)

,country,countryCoded,degree,degreeCoded,eng,engCoded,fieldGroup,fund,gpaBachelors,gpaMasters,gre,highLevelBachUni,highLevelMasterUni,paper,targetUni,uniAccepted,uniBachelors,uniMasters,uniSelected,year
2,united kingdom,4,Doctorate,1,High,1,7,True,3.5,3.442986,False,False,False,8.0,heriot-watt,n/a,azad,n/a,heriot-watt,2007
3,united states,0,Doctorate,1,VeryHigh,0,5,True,2.5,3.500000,True,True,False,13.0,northwestern,switzerland,mashhad,portland state,northwestern,2010


In [17]:
def mydist(x, y):
    return np.sum((x-y)**2)
def jaccard(a, b):
    intersection = float(len(set(a) & set(b)))
    union = float(len(set(a) | set(b)))
    return 1.0 - (intersection/union)

In [18]:
# http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html
    
dist=['braycurtis','canberra','chebyshev','cityblock','correlation','cosine','euclidean','dice','hamming','jaccard','kulsinski','matching','rogerstanimoto','russellrao','sokalsneath','yule']
algorithm=['ball_tree', 'kd_tree', 'brute']

## On country (only MS)

In [19]:
df.columns

Index([u'country', u'countryCoded', u'degree', u'degreeCoded', u'eng',
       u'engCoded', u'fieldGroup', u'fund', u'gpaBachelors', u'gpaMasters',
       u'gre', u'highLevelBachUni', u'highLevelMasterUni', u'paper',
       u'targetUni', u'uniAccepted', u'uniBachelors', u'uniMasters',
       u'uniSelected', u'year'],
      dtype='object')

In [20]:
oldDf=df.copy()
df=df[['countryCoded','degreeCoded','engCoded', 'fieldGroup','fund','gpaBachelors','gre', 'highLevelBachUni', 'paper']]
df=df[df.degreeCoded==0]
del df['degreeCoded']
bestAvg=[]
for alg in algorithm:
    for dis in dist:
        k_fold = KFold(n=len(df), n_folds=5)
        scores = []
        try:
            clf = KNeighborsClassifier(n_neighbors=3, weights='distance',algorithm=alg, metric=dis)
        except Exception as err:
#             print(alg,dis,'err')
            continue
        for train_indices, test_indices in k_fold:
            xtr = df.iloc[train_indices,(df.columns != 'countryCoded')]
            ytr = df.iloc[train_indices]['countryCoded']
            xte = df.iloc[test_indices, (df.columns != 'countryCoded')]
            yte = df.iloc[test_indices]['countryCoded']
            clf.fit(xtr, ytr)
            ypred = clf.predict(xte)
            acc=accuracy_score(list(yte),list(ypred))
            scores.append(acc*100)
        print(alg,dis,np.average(scores))
        bestAvg.append(np.average(scores))
print('>>>>>>>Best: ',np.max(bestAvg))

('ball_tree', 'braycurtis', 33.943345543345544)
('ball_tree', 'canberra', 32.974684574684574)
('ball_tree', 'chebyshev', 31.034106634106632)
('ball_tree', 'cityblock', 33.431501831501834)
('ball_tree', 'euclidean', 33.659422059422056)
('ball_tree', 'dice', 30.746764346764348)
('ball_tree', 'hamming', 33.943671143671146)
('ball_tree', 'jaccard', 30.746764346764348)
('ball_tree', 'kulsinski', 30.861375661375661)
('ball_tree', 'matching', 30.746764346764348)
('ball_tree', 'rogerstanimoto', 30.746764346764348)
('ball_tree', 'russellrao', 35.258608058608061)
('ball_tree', 'sokalsneath', 30.746764346764348)
('kd_tree', 'chebyshev', 32.117867317867322)
('kd_tree', 'cityblock', 33.202930402930399)
('kd_tree', 'euclidean', 33.373544973544981)
('brute', 'braycurtis', 33.316890516890524)
('brute', 'canberra', 32.176475376475381)
('brute', 'chebyshev', 29.949369149369147)
('brute', 'cityblock', 32.746275946275951)
('brute', 'correlation', 31.32242572242572)
('brute', 'cosine', 30.807488807488806)


## On Fund (only MS)

In [21]:
bestAvg=[]
for alg in algorithm:
    for dis in dist:
        k_fold = KFold(n=len(df), n_folds=5)
        scores = []
        try:
            clf = KNeighborsClassifier(n_neighbors=3, weights='distance',algorithm=alg, metric=dis)
        except Exception as err:
            continue
        for train_indices, test_indices in k_fold:
            xtr = df.iloc[train_indices, (df.columns != 'fund')]
            ytr = df.iloc[train_indices]['fund']
            xte = df.iloc[test_indices, (df.columns != 'fund')]
            yte = df.iloc[test_indices]['fund']
            clf.fit(xtr, ytr)
            ypred = clf.predict(xte)
            acc=accuracy_score(list(yte),list(ypred))
            score=acc*100
            scores.append(score)
            if (len(bestAvg)>1) :
                if(score > np.max(bestAvg)) :
                    bestClf=clf
        bestAvg.append(np.average(scores))
        print (alg,dis,np.average(scores))
print('>>>>>>>Best: ',np.max(bestAvg))

('ball_tree', 'braycurtis', 75.868294668294666)
('ball_tree', 'canberra', 75.353846153846149)
('ball_tree', 'chebyshev', 73.245095645095631)
('ball_tree', 'cityblock', 76.668294668294678)
('ball_tree', 'euclidean', 76.098168498168505)
('ball_tree', 'dice', 68.450305250305249)
('ball_tree', 'hamming', 74.898819698819707)
('ball_tree', 'jaccard', 68.450305250305249)
('ball_tree', 'kulsinski', 68.678876678876676)
('ball_tree', 'matching', 68.450305250305249)
('ball_tree', 'rogerstanimoto', 68.450305250305249)
('ball_tree', 'russellrao', 65.768335368335372)
('ball_tree', 'sokalsneath', 68.450305250305249)
('kd_tree', 'chebyshev', 73.359869759869767)
('kd_tree', 'cityblock', 75.526577126577124)
('kd_tree', 'euclidean', 75.298819698819699)
('brute', 'braycurtis', 76.096052096052091)
('brute', 'canberra', 75.696377696377709)
('brute', 'chebyshev', 71.876923076923077)
('brute', 'cityblock', 76.268294668294672)
('brute', 'correlation', 75.297028897028895)
('brute', 'cosine', 74.498494098494092)

### Best : ('brute', 'cityblock', 76.894261294261298)

In [22]:
me=[1,2,0,2.5,False,False,1.5]

In [23]:
n=bestClf.kneighbors([me])
n

(array([[ 0.42857143,  0.42857143,  0.42857143]]),
 array([[ 5, 17, 18]], dtype=int64))

In [24]:
for i in n[1]:
    print(xtr.iloc[i])

    countryCoded  engCoded  fieldGroup  gpaBachelors    gre highLevelBachUni  \
18             1         1           2           2.0  False             True   
71             1         1          10           3.0  False             True   
77             5         1           0           3.0  False             True   

    paper  
18    5.0  
71    1.5  
77    2.5  
